# OscilloDSP

Copyright (c) 2020, 2021, Chubu University and Firmlogics

All rights reserved.

## ドキュメント

[インストール方法](installation.ipynb)、[使い方](usage.ipynb) については、それぞれのリンクを参照してください。
TI Code Composer Studio での、デモアプリの動かし方は、[こちら](ccs_build.ipynb)を参照してください。

## メモ

- うまくグラフが表示されないときは、Jupyter Notebook を再起動すると直ることがあります。

## Jupyter セルの自動スクロールを停止

参考: https://stackoverflow.com/questions/36757301/disable-ipython-notebook-autoscrolling

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## OscilloDSP モジュールのロードと定数の定義

- ```UART_DEVICE```: UART デバイス名
- ```UART_BITRATE```: UART 通信速度

実際の DSP ボードに接続せずに、PC 上で DSP 側の機能をシミュレーションすることができます。
その機能を使う場合には、以下の ```USE_PCSIM``` を```True```にしてください。
この機能は、Linux と Mac OS 上で確認済みです。

```USE_PCSIM``` が ```True``` の場合、```UART_DEVICE``` と ```UART_BITRATE``` は、
後半のセル「PC シミュレータの起動」下の Python コードで上書きされます。

In [ ]:
%matplotlib widget
from oscillodsp import oscillo
from oscillodsp import utils

USE_PCSIM = False  # When using DSP simulator on PC (Linux only)

# UART_DEVICE = "ftdi://ftdi:0xa6d0:TIU72PWC/2"  # Linux
# UART_DEVICE = "ftdi://ftdi:0xa6d0:TIU72PWC/2"  # Mac OS
# UART_DEVICE = "ftdi://ftdi:232:FTGYDM6A/1"  # Mac OS (TTL232R-3V3)
UART_DEVICE = "ftdi://ftdi:0xa6d0:TIU72PWC/2"  # Windows

UART_BITRATE = 2000000
# UART_BITRATE = 115200

## バージョンのチェック

動作確認済みバージョンについては、[こちら](checked_versions.ipynb)を参照してください。

In [ ]:
oscillo.show_module_versions([
    "notebook",
    "matplotlib",
    "ipywidgets",
    "ipympl",
    "serial",
    "pyftdi",
])

## PC シミュレータの起動

In [ ]:
if USE_PCSIM:
    UART_DEVICE = utils.run_pcsim("../pcsim/pcsim")
    UART_BITRATE = 115200
    
# When you need debug pcsim by GDB, run pcsim separately, and run below.
# UART_DEVICE = open("../pcsim/ptyname.txt").read()
# UART_BITRATE = 115200

## オシロのセットアップ

- ```loglevel```と```dsp_loglevel```で、ログ表示レベルを設定できます。
  ログは jupyter を起動したシェル（Windows では cmd.exe）の画面に表示されます。
  ログレベルについては、
  [ここ](https://docs.python.org/3/library/logging.html#logging-levels)
  を参考にしてください。

In [ ]:
import logging

osc = oscillo.Oscillo(
    dsp_tty=UART_DEVICE,
    dsp_bitrate=UART_BITRATE,
    quantize_bits=8,  # reduced for smaller traffic from DSP
    loglevel=logging.INFO,
    dsp_loglevel=logging.INFO)

osc  # Need to evaluate 'osc' here.  Otherwise, screen doesn't show up

## オシロのスタート

最初にオシロを動かすには、以下の関数を実行します。

In [ ]:
osc.start()

## チャネル表示のオンオフ

In [ ]:
osc.view_enabled_ch[0] = True
osc.view_enabled_ch[1] = False
# osc.view_enabled_ch[2] = True
# osc.view_enabled_ch[3] = False

## オシロの終了

終了後に再度実行するには、**「オシロのセットアップ」からやり直してください**。
また、PC シミュレータを使う場合は、「PC シミュレータの起動」からやり直してください。

In [ ]:
osc.close()